In [1]:
import pandas as pd
import tweepy
from tweepy import OAuthHandler
import pandas as pd
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In this notebook, we will get:
- Dataframes of tweets of each president of the party. 
- Dataframes of mentions of users for each president.

### Connecting with Twitter API

In [2]:
%run ./keys.ipynb
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

## 1. PARTIDO POPULAR

In [8]:
test = api.search(q='@populares OR "partido popular" OR "el pp" OR "los del pp"', tweet_mode='extended')
pp_mentions_json = [tweet._json for tweet in test]
df_pp_mentions = pd.json_normalize(pp_mentions_json)

In [13]:
df_pp_mentions['full_text'][0]

'RT @Adrilastra: También estaban Rosa Romero y Alicia García, diputadas del Partido Popular. Y estaban allí porque se trataba de un proyecto…'

In [3]:
pp_mentions = [tweet for tweet in tweepy.Cursor(api.search, 
                                                q='@populares OR "partido popular" OR "el pp" OR "los del pp"', 
                                                lang="es", 
                                                tweet_mode='extended',
                                                result_type="recent").items(500)]

In [4]:
pp_mentions_json = [tweet._json for tweet in pp_mentions]
df_pp_mentions = pd.json_normalize(pp_mentions_json)

In [15]:
df_pp_mentions

,created_at,id,id_str,full_text,truncated,display_text_range,source,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,...,quoted_status.coordinates,quoted_status.place,quoted_status.contributors,quoted_status.is_quote_status,quoted_status.retweet_count,quoted_status.favorite_count,quoted_status.favorited,quoted_status.retweeted,quoted_status.possibly_sensitive,quoted_status.lang
0,Mon Apr 12 19:13:55 +0000 2021,1381687041231097863,1381687041231097863,RT @Adrilastra: También estaban Rosa Romero y ...,False,"[0, 140]","<a href=""http://twitter.com/download/iphone"" r...",NaN,None,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Mon Apr 12 19:13:49 +0000 2021,1381687014244937729,1381687014244937729,RT @PODEMOS: El PP ha utilizado la educación c...,False,"[0, 140]","<a href=""https://mobile.twitter.com"" rel=""nofo...",NaN,None,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Mon Apr 12 19:13:46 +0000 2021,1381687001225781251,1381687001225781251,RT @braulio41554333: El PP ha utilizado la edu...,False,"[0, 140]","<a href=""http://twitter.com/download/android"" ...",NaN,None,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Mon Apr 12 19:13:43 +0000 2021,1381686988089266181,1381686988089266181,RT @Xuxipc: ÚLTIMA HORA: El PP de Madrid susti...,False,"[0, 111]","<a href=""http://twitter.com/download/android"" ...",NaN,None,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Mon Apr 12 19:13:39 +0000 2021,1381686970947096584,1381686970947096584,RT @marmayor: @natalia_velilla Si dais lugar a...,False,"[0, 139]","<a href=""http://twitter.com/#!/download/ipad"" ...",NaN,None,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Mon Apr 12 19:13:36 +0000 2021,1381686958682963973,1381686958682963973,RT @david_bxs: el PP es bochornoso en sí mismo...,False,"[0, 140]","<a href=""https://mobile.twitter.com"" rel=""nofo...",NaN,None,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Mon Apr 12 19:13:31 +0000 2021,1381686937715625984,1381686937715625984,RT @_daniMP_: Apostar e invertir en Educación ...,False,"[0, 140]","<a href=""http://twitter.com/download/android"" ...",NaN,None,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Mon Apr 12 19:13:25 +0000 2021,1381686912243617793,1381686912243617793,RT @maGuardiolaVox: Está cada vez más claro qu...,False,"[0, 140]","<a href=""http://twitter.com/download/iphone"" r...",NaN,None,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Mon Apr 12 19:13:17 +0000 2021,1381686878982844422,1381686878982844422,"@afernb @Cristina_H_ @populares Ostia, acabo d...",False,"[32, 56]","<a href=""http://twitter.com/download/android"" ...",1.381686e+18,1381685945141977092,7.022987e+17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Mon Apr 12 19:13:16 +0000 2021,1381686877384753152,1381686877384753152,RT @ppandaluz: ↗️ El Partido Popular ganaría l...,False,"[0, 140]","<a href=""http://twitter.com/download/android"" ...",NaN,None,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
